In [1]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value

class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

    def times_two(self):
        return self.value * 2

Super 클래스의 Init을 직접적으로 호출하는 것은 예기치 못한 동작을 일으킨다.

MyBaseClass.__init__(self, 5)
 

In [2]:
foo = OneWay(5)
print('First ordering is (5 * 2) + 5 =', foo.value)

NameError: name 'OneWay' is not defined

In [5]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)



NameError: name 'PlusFive' is not defined

In [75]:
bar = AnotherWay(5)
print('Second ordering still is', bar.value)


Second ordering still is 15


이 클래스의 동작은 부모 클래스의 정의한 순서와 일치하지 않으면 문제를 일으킬 수 있다.

여기서 주목할 것은 부모의 초기화 메서드의 순서가 없다는 것이다.





class AnotherWay(MyBaseClass, 1(PlusFive), 2(TimesTwo)):


        2 TimesTwo.__init__(self)
        1 PlusFive.__init__(self)


### 두번째 문제 다이아몬드 상속

In [76]:
class MyBaseClass(object):
    def __init__(self, value):
        print("called MyBaseClass init")
        self.value = value
        
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5

class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2



In [77]:
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)


In [78]:

foo = ThisWay(5)
print('Should be (5 * 5) + 2 = 27 but is', foo.value)

called MyBaseClass init
called MyBaseClass init
Should be (5 * 5) + 2 = 27 but is 7


결과는 27 이어야 하지만, 
두번째 생성자가 호출되면서 value 값을 5로 셋팅한다.
결과적으로 5 + 2 = 7 결과가 도출된다.

### 파이썬 2.2 MRO (Method Resolution Order)
내장함수 super 키워드로 어떤 슈퍼클래스부터 초기화하는지 정의한다.

In [79]:
class MyBaseClass(object):
    def __init__(self, value):
        print("called MyBaseClass init")
        self.value = value

class TimesFiveCorrect(MyBaseClass):
    def __init__(self, value):
        super(TimesFiveCorrect, self).__init__(value)
        self.value *= 5

class PlusTwoCorrect(MyBaseClass):
    def __init__(self, value):
        super(PlusTwoCorrect, self).__init__(value)
        self.value += 2

class GoodWay(TimesFiveCorrect, PlusTwoCorrect):
    def __init__(self, value):
        super(GoodWay, self).__init__(value)

In [80]:
foo = GoodWay(5)
print('Should be 5 * (5 + 2) = 35 and is', foo.value)

called MyBaseClass init
Should be 5 * (5 + 2) = 35 and is 35


다이아몬드 꼭대기인 MyBaseClass 는 이제 1회만 호출된다.

In [81]:

from pprint import pprint
pprint(GoodWay.mro())
pprint = pprint

[<class '__main__.GoodWay'>,
 <class '__main__.TimesFiveCorrect'>,
 <class '__main__.PlusTwoCorrect'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]


MRO 순서는 mro 메서드로 알 수 있다.

파이썬2에서는 주목할 만할 두가지 문제가 있다. 
* 문법이 장황해진다. 

``super(GoodWay, self).__init__(value)``
* super 호출하기 위해서 명시적으로 현재 클래스 이름을 지정해야 한다. 
이는 Refactoring 하는데 취약하다

### 파이썬 3 에서의 super 정의

In [82]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        
class Explicit(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)

class Implicit(MyBaseClass, A, B, C):
    def __init__(self, value):
        super().__init__(value * 2)

assert Explicit(10).value == Implicit(10).value

명시적으로 현재 클래스 이름을 작성하는 것이 아니고 
암시적으로 super 키워드를 통해서 해소할 수 있다.

super().__init__(value * 2)